In [1]:
import kagglehub

path = kagglehub.dataset_download("dev0914sharma/dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/shahwaiz-mughal/.cache/kagglehub/datasets/dev0914sharma/dataset/versions/1


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
df = pd.read_csv("Dataset.csv")

df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['timestamp'], inplace=True)

In [ ]:
scaler = MinMaxScaler()
df['rating'] = scaler.fit_transform(df[['rating']])

df.head()

In [ ]:
user_item_matrix = df.pivot(index="user_id", columns="item_id", values="rating").fillna(0)

user_item_matrix.head()


In [ ]:
user_similarity = cosine_similarity(user_item_matrix)

user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

user_similarity_df.head()
